# Draft data processing

## To do

 - [ ] Clean trajectories (How?). Iterpolation, remove outliers (e.g. points outside path)
 - [ ] Calculate speed for every 1/3 second or something instead of frame-to-frame? It seems like there is a lot of noise (super high speeds), which could be due do video clitches.
 - [x] Anotate video with CSV code. This could be usefull to troubleshoot data.
 

### To do video processing:
 - [ ] Drop tracking if they move out of defined filed of view area. This would solve more or less the issue with tracking different objects as the same

### To do cleaning
- [ ] If a trajectory goes and comes back, split into 2
- [ ] Filter out short trajectories
- [ ] Interpolate frames without tracking if trajectory is monotonic

### To do analysis
 - [ ] Only in trajectory if of a certain class
 - [ ] Intersections

## Settings

In [ ]:
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
from tqdm import tqdm

import os
import copy as cp

In [ ]:
# Import custom functions 
# from utils import *

from modules.utils import *
from modules.draw import *
from modules.homography import PixelMapper

In [ ]:

# PLotting params
plt.rcParams['figure.figsize'] = [15, 10]

# Printing params
pd.set_option("max_rows", 1000)


Functions (move this somewhere else eventually)

In [ ]:
# Plot inline function for one ore two images
def imgshow(img, img2 = None):
    if img2 is None:
        plt.imshow(img)
        # plt.title('World view')
        plt.show()
    else:
        plt.subplot(1, 2, 1)
        plt.imshow(img)


        plt.subplot(1, 2, 2)
        plt.imshow(img2)

        plt.tight_layout()

        plt.show()

        
# Homography plot functions
def draw_point(img, point, color, label = None):
    img_cp = cp.deepcopy(img)
    pcoords = tuple(point)
    cv2.circle(img_cp, pcoords, 4, color, -1)
    if label is not None:
        tcoords = tuple(point + 5)
        cv2.putText(img_cp, label, tcoords,  cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2, cv2.LINE_AA)
    # ishow(img_cp)
    return img_cp


def draw_hom_points(img, points_array, label = True):
    """
    Plots all points in an array in a given image
    """
    
    img_cp = cp.deepcopy(img)
    # Loop over points
    i = 0
    for p in points_array:
        i += 1
        if label:
            label = 'p' + str(i)
        else:
            label = None
        img_cp = draw_point(img_cp, p, (0, 0, 255), label)
    return img_cp


# Draw movements trajectories
def draw_trajectory(img, trajectory_array, color):
    img_cp = cp.deepcopy(img)
    for p in range(1, len(trajectory_array)):
            cv2.line(img_cp, tuple(trajectory_array[p-1]), tuple(trajectory_array[p]), color, 2)
    return img_cp


# Plot areas function
def plot_polygon(*polygons, image, color =(0,255,255), alpha = 0.3, show = True):
    """
    Takes N polygon arrays and plots them on a given image
    """
    
    overlay = cp.deepcopy(image)
    image_new = cp.deepcopy(image)

    for poly in polygons:
        poly_i = cv2.fillPoly(overlay, pts = [poly], color = color)

    image_new = cv2.addWeighted(overlay, alpha, image_new, 1 - alpha, 0)
    
    if show:
        imgshow(image_new)
    else:
        return image_new


In [ ]:
# Filter points from df if in area. Returns a boolean pandas series 
def points_in_area(points_df, area):
    df_dict = points_df[['cx', 'cy']].to_dict('records')
    
    # Empty list to store results
    bol_field_of_view = []
    
    # Loop over all points in df
    for row in tqdm(df_dict):
        # Test if point in area
        p_i = tuple([row['cx'], row['cy']])
        bol_int_i = cv2.pointPolygonTest(area, p_i, False)

        # Convert to boolean (Gambiarra)
        if bol_int_i == -1:
            bol_i = False
        else:
            bol_i = True

        bol_field_of_view.append(bol_i)
    
    return pd.Series(bol_field_of_view)





## Load data

Load tabular data outputed by video-processing model

In [ ]:
os.listdir('data')

In [ ]:
df = pd.read_csv('data/2-sample-30min.csv')

# Calculate centroids
df['cx'] =  round(df['xi'] + (df['xj'] - df['xi'])/2).astype(int)
df['cy'] =  round(df['yi'] + (df['yj'] - df['yi'])/2).astype(int)


Load images data

In [ ]:
img_frame = cv2.imread('data/2-frame.jpg')
img_world = cv2.imread('data/2-sat.jpg')


In [ ]:
imgshow(img_frame, img_world)

In [ ]:
img_frame.shape

## Homography

Homography points defintion: Set the same points in both images and coordinates

In [ ]:
# Create one instance of PixelMapper to convert video frames to coordinates
quad_coords = {
#     # Unprojected
#     "lonlat": np.array([
#         [9.035947, 38.853956], # P1 top right
#         [9.035794, 38.853338], # P2 top left
#         [9.035631, 38.853425], # P3 bottom left
#         [9.035782, 38.853971] #  P4 bottom right
#     ]),
    # UTM
    "lonlat": np.array([
        [998829.59, 483949.29], # P1 top right
        [998812.71, 483881.37], # P2 top left
        [998794.68, 483890.92], # P3 bottom left
        [998811.35, 483950.93] #  P4 bottom right
    ]),
    "pixel": np.array([
        [426, 37], #  P1 top right
        [119, 121], #  P2 top left
        [547, 171], # P3 bottom left
        [596, 50] # P4 bottom right
    ]),
    "pixel_sat": np.array([
        [648, 322], # P1 top right
        [295, 402], # P2 top left
        [322, 488], #  P3 bottom left
        [661, 413] # P4 bottom right
    ])
}

# Display image points. 
img_s_points = draw_hom_points(img_world, quad_coords['pixel_sat'])
img_f_points = draw_hom_points(img_frame, quad_coords['pixel'])
imgshow(img_f_points, img_s_points)

PixelMapper class converts coordinates in pixels to lat-long

In [ ]:
# Create pixel maper instance to convert from video to lat long (and vice versa)
pm = PixelMapper(quad_coords["pixel"], quad_coords["lonlat"])

# Create pixel maper instance to convert from sat image to lat long (and vice versa)
pm_sat = PixelMapper(quad_coords["pixel_sat"], quad_coords["lonlat"])

Test trajectory to see if homography is working:

In [ ]:
# To do
# - Make sure trajectories are in frame order
# - Organize this into a function






# Create trajectory df
car_df = df[df['obj_id'] == 197]

t_car = car_df[['cx', 'cy']].to_numpy()

# Anotate trajectory on initial video frame
img_cf = img_frame.copy()
img_cf = draw_trajectory(img_cf, t_car, (0, 0, 255))

# # ishow(img_cf)

# Transform trajectories to long lat
t_car_ll = pm.pixel_to_lonlat(t_car) # t_car created in draft-intersections.py

# Transform lat long trajectory into pixels of sat image
t_car_s = pm_sat.lonlat_to_pixel(t_car_ll).astype(int)

# # Anotate trajectory on sat image
img_cs = img_world.copy()
img_cs = draw_trajectory(img_cs, t_car_s, (0, 0, 255))

imgshow(img_cf, img_cs)

In [ ]:
car_df

In [ ]:
fig = px.scatter(car_df, x="cx", y="cy", text="frame", log_x=True, size_max=60)
fig.show()

## Frame definition

Set boundaries to exclude objects too far away from the camera since tracking tend to be less precise.

In [ ]:
# Create new objects in memory
bound_overlay = cp.deepcopy(img_frame)
bound_image = cp.deepcopy(img_frame)



# Frame boundaries
boundaries = np.array([
    # Camera corners
    [0,0], # top left
    [0,img_frame.shape[0]], # bottom left
    [img_frame.shape[1],img_frame.shape[0]], # bottom right
    
    # Additional points to determine limits RIGHT to LEFT
    [700,100],
    [600,55],
    [430,25],
    [250,30],

    ]) 

alpha = 0.2  # Transparency factor.


# Camera field of view polygon
bound_poly = cv2.fillPoly(bound_overlay, pts = [boundaries], color =(255,0,0))


# PLot field of view
bound_image = cv2.addWeighted(bound_overlay, alpha, bound_image, 1 - alpha, 0)
bound_image = draw_hom_points(bound_image, boundaries, label = False)


imgshow(bound_image)



Filter data frame to only points within field of view

In [ ]:
# Test each point individually. Try a more efficient way!

# image_new_cp = cp.deepcopy(bound_image)


# Supposedly the most efficient way of looping through a pandas df
df_dict = df.to_dict('records')


# Empty list to store results
bol_field_of_view = []

# Loop over all points in df
for row in tqdm(df_dict):
    # Test if point in area
    p_i = tuple([row['cx'], row['cy']])
    bol_int_i = cv2.pointPolygonTest(boundaries, p_i, False)
    
    # Convert to boolean (Gambiarra)
    if bol_int_i == -1:
        bol_i = False
    else:
        bol_i = True
    
    bol_field_of_view.append(bol_i)
#     print(bol_i)
#     cv2.circle(image_new_cp, tuple(p_i), radius=1, color=(0, 0, 255), thickness=3)    




In [ ]:


df[pd.Series(bol_field_of_view)].shape,  df.shape
# bool(bol_field_of_view)
# # bol_field_of_view
# type(bol_field_of_view)
# bol_field_of_view.astype('bool')

# bol_field_of_view

In [ ]:
371069/592934

## Movements definitions

Define preliminary east and west through movement. For now, not making the distinction for direction for simplicity

In [ ]:
# First trajectory area
contours1 = np.array([[5,190], # top left 
                     [350,480], # bottom left
                     [400,430], # bottom right
                     [50,170]]) # top right
# Second trajectory area
contours2 = np.array([[430,40], # top left 
                     [580,100], # bottom left
                     [593,70], # bottom right
                     [470,35]]) # top right

plot_polygon(contours1, contours2, image = img_frame)

Define preliminary crossing pedestrians movement. Also not caring about order initially, but might increase precision to define multiple areas and define it as crossing any opposite pair in order.

In [ ]:


# First trajectory area
ped_area_1 = np.array([[5,170], # top left 
                       [400,35], # bottom left
                       [425,50], # bottom right
                       [5,210]]) # top right

# Second trajectory area
ped_area_2 = np.array([[270,480], # top left 
                         [550,70], # bottom left
                         [580,90], # bottom right
                         [370,480]]) # top right

plot_polygon(ped_area_1, ped_area_2, image = img_frame)    


# foo = plot_polygon(ped_area_1, ped_area_2, image = img_frame , show = False)
# foo = draw_hom_points(foo,ped_area_2)
# imgshow(foo)


Filter only trajectories that match the movement (o try to)

In [ ]:
# Test trajectory

trj_temp = df[df['obj_id'] == 15]; trj_temp

p_temp = tuple(trj_temp.iloc[2][['cx','cy']]); p_temp

Test if any points in the trajectory are in the defined areas. Idea is that if there are points in both, that trajectory is part of a pre defined movement.

In [ ]:

# Copy image to anotate with trajectory
image_new_cp = cp.deepcopy(img_frame)

# Test each point individually. Try a more efficient way!


bol_area1 = []
bol_area2 = []

for i in range(0, len(trj_temp)):
    p_i = tuple(trj_temp.iloc[i][['cx', 'cy']])
    
    # Test if points in area
    bol_area1.append(cv2.pointPolygonTest(contours1, p_i, False))
    bol_area2.append(cv2.pointPolygonTest(contours2, p_i, False))

    cv2.circle(image_new_cp, tuple(p_i), radius=1, color=(0, 0, 255), thickness=3)
#     print(p_i)
#     print(bol)

# Test if any point in area
any(bol_area1)

any(bol_area2)

In [ ]:
imgshow(image_new_cp)


# Filter trajectories

In [ ]:
# df[points_in_area(df, ped_area_1)].shape[0]/df.shape[0]

def filter_by_movement(area1, area2, df):
    """
    Filter data frame by trajectories that are part of a movement. Defined by 
    having a at lease one point in both areas.
    """
    
    # Bool columns
    df['area1'] = points_in_area(df, area1)
    df['area2'] = points_in_area(df, area2)

    
    # Get ids of movements
    df_agg = df\
        .groupby('obj_id')\
        .agg({'area1' : any, 'area2' : any})
    
    trajectories_in_movement = df_agg[df_agg['area1'] & df_agg['area2']].index
    
    return df[df['obj_id'].isin(trajectories_in_movement)]


mov1 = filter_by_movement(ped_area_1, ped_area_2, df)
mov2 = filter_by_movement(contours1, contours2, df)



In [ ]:
mov2['obj_id'].unique()

In [ ]:

df_agg = df_cp\
    .groupby('obj_id')\
    .agg({'area1' : any, 'area2' : any})





# len(trajectories_in_movement)

# def filter_movements(areas, df):

In [ ]:
# ?? 75% of all points are in area 1???

# df[points_in_area(df, ped_area_1)].shape[0]/df.shape[0]
# 0.7547146899992242


##  Calculate point speed

In [ ]:
# Test distance between reference points to see if consistent

In [ ]:
quad_coords['lonlat'][0]

In [ ]:
quad_coords['lonlat'][1]

In [ ]:
998829.59-998812.71

In [ ]:

# math.dist(quad_coords['lonlat'][0], quad_coords['lonlat'][1])

# quad_coords['lonlat'][0] - quad_coords['lonlat'][1]
np.linalg.norm(quad_coords['lonlat'][0] - quad_coords['lonlat'][1])


In [ ]:
# Create trajectory df
trj_temp = df[df['obj_id'].isin([1,15])]
# trj_temp = df.copy()

# trj_temp = df[df['obj_id'].isin([15])]


t_temp = trj_temp[['cx', 'cy']].to_numpy()


# # ishow(img_cf)

# Transform trajectories to long lat
t_temp_ll = pm.pixel_to_lonlat(t_temp) # t_car created in draft-intersections.py

In [ ]:
foo = pd.DataFrame(t_temp_ll)
foo.columns = ['c_lat', 'c_long']
# foo

In [ ]:
new_df = pd.concat([trj_temp.reset_index(drop=True), foo.reset_index(drop=True)], axis=1)
new_df = new_df.sort_values(['obj_id', 'frame'])

In [ ]:
# lag variable
# shifted = new_df.groupby("obj_id") #.shift(1) 
# shifted = new_df.shift(1)[['frame', 'obj_id', 'c_lat', 'c_long']]


new_df['c_lat_l'] = new_df.groupby(['obj_id'])['c_lat'].shift(1)
new_df['c_long_l'] = new_df.groupby(['obj_id'])['c_long'].shift(1)

new_df


# df_new.join(shifted, rsuffix = '_l')
# new_df.shape

In [ ]:
# Calculate distance from point to point lag for the whole data

from scipy.spatial.distance import cdist

foo = cdist(new_df[['c_lat', 'c_long']], new_df[['c_lat_l', 'c_long_l']], 'euclid')
new_df['dist_m'] = np.diagonal(foo)

new_df

In [ ]:


# Calculate speed
fps = 30

new_df['speed_ms'] = new_df['dist_m']/(1/fps)
new_df['speed_kmh'] = new_df['speed_ms']*3.6
new_df[['frame', 'obj_id', 'cx', 'cy', 'c_lat','c_long', 'c_lat_l', 'c_long_l','dist_m', 'speed_ms','speed_kmh']]




In [ ]:
new_df.columns

In [ ]:
# Comaparing two functions
# np.linalg.norm(np.array([998792.072463,483877.645084]) - np.array([998792.089436, 483877.619475]))

In [ ]:
t_temp_ll.shape

In [ ]:
trj_temp

# DRAFT

In [ ]:
#----------------------------------------------------
# Use mouse to find points (or try to at least)


# Marker and grab position function

img = cp.deepcopy(img_frame)
mouseX,mouseY = 0,0

def draw_circle(event,x,y,flags,param):
    global mouseX,mouseY
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img,(x,y),3,(255,0,0),1)
        mouseX,mouseY = x,y
        

# A window that will capture the mouse click
# cv2.namedWindow('image')
# cv2.setMouseCallback('image', draw_circle)

# while(1):
#     cv2.imshow('image',img)
#     # Kill window if Q is pressed
#     k = cv2.waitKey(1) & 0xFF
#     if k == ord('q'):
#         cv2.destroyAllWindows()
#         break
#     elif k == ord('a'):
#         print(mouseX,mouseY)
# print(mouseX,mouseY)

In [ ]:
cv2.destroyAllWindows()